In [1]:
%%capture --no-display
%pip install python-dotenv
%pip install web3

In [2]:
%load_ext dotenv
%dotenv

In [3]:
import os

infuraApiKey = os.environ.get("INFURA_API_KEY")

In [4]:
from web3 import Web3

infura_url = f"https://polygon-mainnet.infura.io/v3/{infuraApiKey}"
web3 = Web3(Web3.HTTPProvider(infura_url))
web3.is_connected()

True

Polygon USDC contract
- https://docs.compound.finance/#developer-resources 
- https://polygonscan.com/address/0xf25212e676d1f7f89cd72ffee66158f541246445

In [5]:
import json

sc_address = "0xF25212E676D1F7F89Cd72fFEe66158f541246445"
with open ("./abi/compound_v3_usdc.json") as file:
    abi = json.load(file)

contract = web3.eth.contract(address=sc_address, abi=abi)
contract.all_functions()

[<Function absorb(address,address[])>,
 <Function accrueAccount(address)>,
 <Function approveThis(address,address,uint256)>,
 <Function balanceOf(address)>,
 <Function baseBorrowMin()>,
 <Function baseMinForRewards()>,
 <Function baseScale()>,
 <Function baseToken()>,
 <Function baseTokenPriceFeed()>,
 <Function baseTrackingBorrowSpeed()>,
 <Function baseTrackingSupplySpeed()>,
 <Function borrowBalanceOf(address)>,
 <Function borrowKink()>,
 <Function borrowPerSecondInterestRateBase()>,
 <Function borrowPerSecondInterestRateSlopeHigh()>,
 <Function borrowPerSecondInterestRateSlopeLow()>,
 <Function buyCollateral(address,uint256,uint256,address)>,
 <Function decimals()>,
 <Function extensionDelegate()>,
 <Function getAssetInfo(uint8)>,
 <Function getAssetInfoByAddress(address)>,
 <Function getBorrowRate(uint256)>,
 <Function getCollateralReserves(address)>,
 <Function getPrice(address)>,
 <Function getReserves()>,
 <Function getSupplyRate(uint256)>,
 <Function getUtilization()>,
 <Funct

In [6]:
decimals = contract.functions.decimals().call()
decimals

6

In [7]:
asset_number = contract.functions.numAssets().call()
asset_number

5

In [8]:
baseAsset = contract.functions.baseToken().call()
baseAsset

'0x2791Bca1f2de4661ED88A30C99A7a9449Aa84174'

In [9]:
for x in range (0, asset_number):
    print (contract.functions.getAssetInfo(x).call())
    print ('collateral_order', contract.functions.getAssetInfo(x).call()[0])
    print ('collateral_token', contract.functions.getAssetInfo(x).call()[1])
    print ('chainlink_price_feed', contract.functions.getAssetInfo(x).call()[2])
    print ('---------------------------------------------------------------')

(0, '0x7ceB23fD6bC0adD59E62ac25578270cFf1b9f619', '0xF9680D99D6C9589e2a93a78A04A279e509205945', 1000000000000000000, 800000000000000000, 850000000000000000, 950000000000000000, 20000000000000000000000)
collateral_order 0
collateral_token 0x7ceB23fD6bC0adD59E62ac25578270cFf1b9f619
chainlink_price_feed 0xF9680D99D6C9589e2a93a78A04A279e509205945
---------------------------------------------------------------
(1, '0x1BFD67037B42Cf73acF2047067bd4F2C47D9BfD6', '0xDE31F8bFBD8c84b5360CFACCa3539B938dd78ae6', 100000000, 750000000000000000, 850000000000000000, 900000000000000000, 150000000000)
collateral_order 1
collateral_token 0x1BFD67037B42Cf73acF2047067bd4F2C47D9BfD6
chainlink_price_feed 0xDE31F8bFBD8c84b5360CFACCa3539B938dd78ae6
---------------------------------------------------------------
(2, '0x0d500B1d8E8eF31E21C99d1Db9A6444d3ADf1270', '0xAB594600376Ec9fD91F8e885dADF0CE036862dE0', 1000000000000000000, 650000000000000000, 800000000000000000, 900000000000000000, 20000000000000000000000000

In [10]:
collateral0 = '0x7ceB23fD6bC0adD59E62ac25578270cFf1b9f619' # Wrapped Ether
collateral1 = '0x1BFD67037B42Cf73acF2047067bd4F2C47D9BfD6' # Wrapped Bitcoin
collateral2 = '0x0d500B1d8E8eF31E21C99d1Db9A6444d3ADf1270' # WMatic

base_token = contract.functions.baseToken().call()
collateral_list = [collateral0, collateral1, collateral2, base_token]
tvl = {}

with open ("./abi/erc20.json") as file:
    erc_abi = json.load (file)

for collateral in collateral_list:
    collateralcontract = web3.eth.contract(address=collateral, abi=erc_abi)
    d = collateralcontract.functions.decimals().call()
    locked = (collateralcontract.functions.balanceOf(sc_address).call()) * 10 ** -(int(d))
    print (collateralcontract.functions.name().call(), d, locked)
    tvl[collateralcontract.functions.name().call()] = locked

Wrapped Ether 18 1209.8417886229417
(PoS) Wrapped BTC 8 280.48327759
Wrapped Matic 18 13494274.025718307
USD Coin (PoS) 6 2797723.2846709997


In [11]:
import requests

url = "https://api.coingecko.com/api/v3/simple/price?ids=bitcoin&vs_currencies=usd"
btc_price = requests.get(url).json()["bitcoin"]["usd"]

url = "https://api.coingecko.com/api/v3/simple/price?ids=ethereum&vs_currencies=usd"
eth_price = requests.get(url).json()["ethereum"]["usd"]

url = "https://api.coingecko.com/api/v3/simple/price?ids=matic-network&vs_currencies=usd"
matic_price = requests.get(url).json()["matic-network"]["usd"]

url = "https://api.coingecko.com/api/v3/simple/price?ids=usd-coin&vs_currencies=usd"
usdc_price = requests.get(url).json()["usd-coin"]["usd"]

In [12]:
prices_dic = {
  '(PoS) Wrapped BTC':btc_price, 
  'Wrapped Ether': eth_price,
  'Wrapped Matic': matic_price,
  'USD Coin (PoS)': usdc_price
}
prices_dic

{'(PoS) Wrapped BTC': 69190,
 'Wrapped Ether': 3754.83,
 'Wrapped Matic': 0.732305,
 'USD Coin (PoS)': 1.0}

In [13]:
import pandas as pd

df = pd.DataFrame.from_dict(tvl, orient='index', columns=['token_amount'])
df['price'] = df.index.map(prices_dic)
df.head()

,token_amount,price
Wrapped Ether,1.209842e+03,3754.830000
(PoS) Wrapped BTC,2.804833e+02,69190.000000
Wrapped Matic,1.349427e+07,0.732305
USD Coin (PoS),2.797723e+06,1.000000


In [14]:
df['amount_usd'] = df['token_amount'] * df['price']
tvl = df.amount_usd.sum()
tvl

36629035.84470183

In [15]:
total_collateral = df['amount_usd'].iloc[:-1].sum()
total_collateral

33831312.560030825

In [16]:
total_borrow = contract.functions.totalBorrow().call() * 10 ** -int(decimals)
total_borrow

14263946.949843999

In [17]:
loan_to_deposit_ratio = (total_borrow / total_collateral) * 100
loan_to_deposit_ratio

42.16196733287785

In [18]:
total_earning = contract.functions.totalSupply().call() * 10 ** -int(decimals)
total_earning

16776940.919552999

In [19]:
utilization = contract.functions.getUtilization().call() * 10 ** -18
utilization * 100

85.02106282636129

In [20]:
contract.functions.getReserves().call() * 10 ** -decimals

284729.392598